### Model API Workflow

1. CloudWatch Events로부터 호출 받을 때, 시간을 보내거나, image 내에서 현재 시간 얻어내거나. (난이도로 봤을 때 훨씬 쉬운건 아무것도 안보내고, image 내에서 진행하는 것.)

2. 시간을 20분 단위로 내림하기.

3. RDS로부터 내림한 시간을 기준으로 데이터 받음. (evs info & timestamp's sequence(length: n) & pymysql)

4. 받은 데이터 utils.py 돌리고 model 실행.

5. RDS로부터 데이터 전달하여 update.

### 내림 시간 계산

In [55]:
from datetime import datetime

def get_dt():
    now = datetime.now()
    dt = datetime(now.year, now.month, now.day, now.hour, now.minute//20*20, 00)
    dt = dt.strftime('%Y-%m-%d %H:%M:%S')
    return dt

get_dt()

'2023-08-13 12:40:00'

### DB 연동 및 데이터 입력

In [57]:
import pymysql
import pandas as pd

attributes = pd.read_csv('data/evcs_attributes.csv')
embedding = pd.read_csv('data/evcs_embedding.csv')

In [58]:
connection = pymysql.connect(
    host=host,
    user=user,
    password=password,
    db=db,
    charset='utf8',
    port = 3306
)

In [59]:
# Create Table
create_query = '''
CREATE TABLE IF NOT EXISTS station (
Sid INT NOT NULL PRIMARY KEY, 
Sname VARCHAR(100) NOT NULL, 
Latitude DOUBLE NOT NULL, 
Longitude DOUBLE NOT NULL, 
Capacity FLOAT NOT NULL, 
Slow_Chargers INT NOT NULL,
Fast_Chargers INT NOT NULL,
Mean_trip DOUBLE NOT NULL,
Length_city DOUBLE NOT NULL,
Length_highway DOUBLE NOT NULL,
Length_local DOUBLE NOT NULL,
Length_national DOUBLE NOT NULL,
Indust_ratio DOUBLE NOT NULL,
Etc_ratio DOUBLE NOT NULL,
Green_ratio DOUBLE NOT NULL,
Commerce_ratio DOUBLE NOT NULL,
Reside_ratio DOUBLE NOT NULL,
UMAP_1 DOUBLE NOT NULL,
UMAP_2 DOUBLE NOT NULL,
UMAP_3 DOUBLE NOT NULL,
UMAP_4 DOUBLE NOT NULL,
UMAP_5 DOUBLE NOT NULL,
UMAP_6 DOUBLE NOT NULL,
UMAP_7 DOUBLE NOT NULL,
UMAP_8 DOUBLE NOT NULL
)
'''

with connection.cursor() as cursor:
    cursor.execute(create_query.replace('\n', ''))

In [60]:
# Insert Values into Database
for idx in range(attributes.shape[0]):
    atts = attributes.iloc[idx, :].values.tolist()
    emb = embedding.iloc[idx, :].values.tolist()[1:]
    values = tuple([idx] + atts + emb)
    insert_query = "INSERT INTO station VALUES {};"
    
    with connection.cursor() as cursor:
        cursor.execute(insert_query.format(values))

connection.commit()
connection.close()

### DB 가져오기

In [1]:
import pymysql
import pandas as pd

In [8]:
connection = pymysql.connect(
    host=host,
    user=user,
    password=password,
    db=db,
    charset='utf8',
    port = 3306
)

select_query = "SELECT * FROM station"

with connection.cursor() as cursor:
    cursor.execute(select_query)
    result = cursor.fetchall()

In [4]:
atts_cols = ['Latitude', 'Longitude', 'Capacity', 'Slow_Chargers',
       'Fast_Chargers', 'Mean_trip', 'Length_city', 'Length_highway',
       'Length_local', 'Length_national', 'Indust_ratio', 'Etc_ratio',
       'Green_ratio', 'Commerce_ratio', 'Reside_ratio']

embed_cols = ['UMAP_1', 'UMAP_2', 'UMAP_3', 
        'UMAP_4', 'UMAP_5', 'UMAP_6', 'UMAP_7', 'UMAP_8']

In [5]:
df = pd.DataFrame(result, columns=[col[0] for col in cursor.description])

atts = df.loc[:, atts_cols]
embed = df.loc[:, embed_cols]

In [6]:
from utils import json2tensor
Sids, X = json2tensor(atts.to_dict('index'))

In [7]:
X

tensor([[1.2891e+02, 3.7744e+01, 5.0000e+01,  ..., 4.3682e-02, 0.0000e+00,
         9.5632e-01],
        [1.2890e+02, 3.7755e+01, 5.0000e+01,  ..., 2.6380e-01, 3.2426e-01,
         4.1194e-01],
        [1.2891e+02, 3.7754e+01, 5.0000e+01,  ..., 2.7188e-01, 0.0000e+00,
         7.2812e-01],
        ...,
        [1.2792e+02, 3.7343e+01, 5.0000e+01,  ..., 7.2788e-01, 1.5569e-01,
         1.1643e-01],
        [1.2934e+02, 3.6019e+01, 5.0000e+01,  ..., 3.3261e-01, 1.2972e-02,
         6.5442e-01],
        [1.2773e+02, 3.5521e+01, 5.0000e+01,  ..., 1.2440e-01, 1.8324e-01,
         6.9236e-01]])

### DB 입력

In [2]:
def update_occupancy(y_json):

    connection = pymysql.connect(
    host=host,
    user=user,
    password=password,
    db=db,
    charset='utf8',
    port = 3306
)

    create_query = '''
CREATE TABLE occupancy (
Sid INT NOT NULL PRIMARY KEY, 
Occupancy_20 INT NOT NULL,
Occupancy_60 INT NOT NULL,
Occupancy_120 INT NOT NULL,
FOREIGN KEY (sid) REFERENCES station(sid)
)
'''
    select_query = "SELECT sid FROM station"

    insert_query = "INSERT INTO station VALUES {};"

    update_query = "UPDATE occupancy SET Occupancy_20 = %d Occupancy_60 = %d Occupancy_120 = %d WHERE Sid = %d"

    try:
        with connection.cursor() as cursor:
                cursor.execute(create_query.replace('\n', ''))
                cursor.execute(select_query)
                result = cursor.fetchall()

        insert_query = "INSERT INTO occupancy VALUES {};"
        for idx in result:
            values = tuple(list(idx) + y_json[idx[0]])
            with connection.cursor() as cursor:
                cursor.execute(insert_query.format(values))
            
        connection.commit()
        connection.close()

    except:
        with connection.curosr() as cursor:
            cursor.execute(update_query, tuple(y_json[idx[0]], list(idx)))


### Keep in mind
- model, input이 제대로 주어지면 utils.py, inference.py 수정
- 현재 시간 기준으로 가져올 것이기 때문에 Table에 데이터 입력시 2023년 8~10월로 맞춰야함.